In [27]:
import polars as pl
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

In [28]:
data_folder = Path("../data/predict-energy-behavior-of-prosumers")

files = {
    "train": "train.csv",
    "gas_prices": "gas_prices.csv",
    "client": "client.csv",
    "electricity_prices": "electricity_prices.csv",
    "forecast_weather": "forecast_weather.csv",
    "historical_weather": "historical_weather.csv",
}

In [29]:
train_df = pl.read_csv(
    data_folder / files["train"]
)

gas_df = pl.read_csv(
    data_folder / files["gas_prices"]
)

client_df = pl.read_csv(
    data_folder / files["client"]
)

electricity_df = pl.read_csv(
    data_folder / files["electricity_prices"]
)

forecast_df = pl.read_csv(
    data_folder / files["forecast_weather"]
)

historical_df = pl.read_csv(
    data_folder / files["historical_weather"]
)

In [30]:
train_df.head(10)

county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
i64,i64,i64,f64,i64,str,i64,i64,i64
0,0,1,0.713,0,"""2021-09-01 00:00:00""",0,0,0
0,0,1,96.59,1,"""2021-09-01 00:00:00""",0,1,0
0,0,2,0.0,0,"""2021-09-01 00:00:00""",0,2,1
0,0,2,17.314,1,"""2021-09-01 00:00:00""",0,3,1
0,0,3,2.904,0,"""2021-09-01 00:00:00""",0,4,2
0,0,3,656.859,1,"""2021-09-01 00:00:00""",0,5,2
0,1,0,0.0,0,"""2021-09-01 00:00:00""",0,6,3
0,1,0,59.0,1,"""2021-09-01 00:00:00""",0,7,3
0,1,1,0.0,0,"""2021-09-01 00:00:00""",0,8,4


## prediction_unit_id
El objetivo es saber como funciona la columna "prediction_unit_id" para ello 
se crea un nuevo dataframe formado solo por esta columna y aquellas por las que está formada

In [31]:
pu_id = train_df.select(pl.col(
    "county", "is_business", "product_type", "is_consumption", "prediction_unit_id"
    ))
pu_id.head(10)

county,is_business,product_type,is_consumption,prediction_unit_id
i64,i64,i64,i64,i64
0,0,1,0,0
0,0,1,1,0
0,0,2,0,1
0,0,2,1,1
0,0,3,0,2
0,0,3,1,2
0,1,0,0,3
0,1,0,1,3
0,1,1,0,4


In [32]:
pu_id[10:20]

county,is_business,product_type,is_consumption,prediction_unit_id
i64,i64,i64,i64,i64
0,1,3,0,5
0,1,3,1,5
1,0,1,0,6
1,0,1,1,6
1,0,3,0,7
1,0,3,1,7
1,1,3,0,8
1,1,3,1,8
2,0,3,0,9


Se obtienen los valores únicos de cada columna

In [35]:
for col in pu_id.columns:
    print(col, ":\n\t", pu_id[col].unique().to_list())

county :
	 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
is_business :
	 [0, 1]
product_type :
	 [0, 1, 2, 3]
is_consumption :
	 [0, 1]
prediction_unit_id :
	 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68]


Puede observarse que existen 15 condados, un valor booleano para determinar 
si el cliente es un negócio y 4 tipos de productos

observando el DataFrame puede observarse que prediction_unit_id  
sigue un orden en el cual, dentro de cada condado, primero se  
ordena por "is_business" y luego por "product_type".  

De esta forma que el orden de prioridad sería:
county, is_business y product_type.

por ejemplo: 

12, 0, 0, 30  
12, 0, 2, 31  
13, 1, 0, 32  
13, 1, 1, 33  
14, 0, 1, 34  

De esta forma "prediction_unit_id" sería un identificador único
que permite unificar a diferentes clientes con el mismo 
contrato dentro de un mismo condado con el fin de unificar
las predicciones.